In [1]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [2]:
%%writefile reducer.py

import sys


current_key = None
word_sum = 0

Overwriting reducer.py


In [3]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


In [4]:
%%writefile reducer2.py

import sys
import collections

current_key = None
word_sum = 0


Overwriting reducer2.py


In [5]:
%%writefile -a reducer2.py


diction = {}
for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            diction[current_key] = word_sum
            if current_key == "the":
                spare_key = current_key
    
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    diction[current_key] = word_sum
new_dict = sorted(diction.items(), key=lambda x: x[1],reverse=True)
#print "%s\t%d" % ('the', diction['the'])
for k in new_dict:
    print "%s\t%d" % (k[0], k[1])



Appending to reducer2.py


In [6]:
! hdfs dfs -ls /data/wiki

Found 1 items
drwxrwxrwx   - jovyan supergroup          0 2017-10-17 13:15 /data/wiki/en_articles_part


In [7]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=2

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,reducer2.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer2.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null
hdfs dfs -cat ${OUT_DIR}/part-00000  ${OUT_DIR}/part-00001 | sort -nrk2| head -7 | tail -1


is	126420


rm: `wordcount_result_1513789754187530': No such file or directory
17/12/20 17:09:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/20 17:09:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/20 17:09:18 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/20 17:09:18 INFO mapreduce.JobSubmitter: number of splits:2
17/12/20 17:09:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1513735525749_0053
17/12/20 17:09:18 INFO impl.YarnClientImpl: Submitted application application_1513735525749_0053
17/12/20 17:09:19 INFO mapreduce.Job: The url to track the job: http://f6bf87a81fd4:8088/proxy/application_1513735525749_0053/
17/12/20 17:09:19 INFO mapreduce.Job: Running job: job_1513735525749_0053
17/12/20 17:09:24 INFO mapreduce.Job: Job job_1513735525749_0053 running in uber mode : false
17/12/20 17:09:24 INFO mapreduce.Job:  map 0% reduce 0%
17/12/20 17:09:40 INFO mapreduce.Job:  map 30% reduce 0%
17/12/20 17:0